In [ ]:
!pip install -q diffusers accelerate fastapi pyngrok nest-asyncio timm mediapipe uvicorn peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 k

In [ ]:
import torch
from fastapi import FastAPI, HTTPException, File, UploadFile
from pydantic import BaseModel
from io import BytesIO
import base64
from pyngrok import ngrok
import uvicorn
import nest_asyncio
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
import timm
import mediapipe as mp
import cv2
from PIL import Image
from diffusers import DiffusionPipeline
import numpy as np
import io
from google.colab import drive
drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Mounted at /content/drive


In [ ]:
model_id = "aldente0630/musinsaigo-3.0"
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16
)
pipe = pipe.to(device)
pipe.load_lora_weights(model_id)
expression_model = torch.load("/content/drive/MyDrive/best_model.pt", map_location=device)
gender_model=torch.load('/content/drive/MyDrive/swinv2_gender.pt',map_location=device)
gender_model.eval()
gender_model.to(device)
expression_model.eval()
expression_model.to(device)
label = ['angry', 'happy', 'panic', 'sad']
gender = ['man', 'woman']

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
mp_face_detection = mp.solutions.face_detection
def detect_and_crop_face(image):
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        image_np = np.array(image)
        results = face_detection.process(image_np)
        if results.detections:
            detection = results.detections[0]
            bbox = detection.location_data.relative_bounding_box
            ih, iw, _ = image_np.shape
            xmin = int(bbox.xmin * iw)
            ymin = int(bbox.ymin * ih)
            width = int(bbox.width * iw)
            height = int(bbox.height * ih)
            xmax = xmin + width
            ymax = ymin + height
            face = image.crop((xmin, ymin, xmax, ymax))
            return face
        else:
            return None
expression_test_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
gender_test_transform=Compose([
    Resize((256,256)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
def make_prompt(prompt: str) -> str:
    prompt_prefix = "RAW photo"
    prompt_suffix = "(high detailed skin:1.2), 8k uhd, dslr, soft lighting, high quality, film grain, Fujifilm XT3"
    return ", ".join([prompt_prefix, prompt, prompt_suffix]).strip()


def make_negative_prompt(negative_prompt: str) -> str:
    negative_prefix = "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), \
    text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, \
    extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, \
    bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, \
    extra arms, extra legs, fused fingers, too many fingers, long neck"

    return (
        ", ".join([negative_prefix, negative_prompt]).strip()
        if len(negative_prompt) > 0
        else negative_prefix
    )

In [ ]:
from fastapi import FastAPI, File, UploadFile, HTTPException
from pydantic import BaseModel
from PIL import Image, UnidentifiedImageError
import io
import torch
import base64
from typing import Optional
from pyngrok import ngrok
import uvicorn
import nest_asyncio
app = FastAPI()

class ImageRequest(BaseModel):
    prompt: str
    negative_prompt: str = ""
    num_inference_steps: int = 28
    guidance_scale: float = 7.0

@app.post("/fashion")
async def fashion(file: UploadFile = File(...)):
    try:
        contents = await file.read()
        image = Image.open(io.BytesIO(contents))
    except UnidentifiedImageError:
        raise HTTPException(status_code=400, detail="Unable to identify the image file. Please upload a valid image.")
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred while processing the image: {str(e)}")

    try:
        face = detect_and_crop_face(image)
        expression_img = expression_test_transform(face)
        gender_img = gender_test_transform(face)
        expression_img = expression_img.unsqueeze(0).to(device)
        gender_img = gender_img.unsqueeze(0).to(device)

        with torch.no_grad():
            expression_pred = expression_model(expression_img)
            gender_pred = gender_model(gender_img)

        expression_pred = label[expression_pred.argmax(1).detach().cpu().numpy().tolist()[0]]
        gender_pred = gender[(torch.sigmoid(gender_pred) > 0.5).int().detach().cpu().numpy().tolist()[0][0]]

        prompt = f"A Korean {gender_pred} {expression_pred} mood in her 20s wearing in street fashion"
        NEGATIVE_PROMPT = ""

        image = pipe(
            prompt=make_prompt(prompt),
            height=1024,
            width=768,
            num_inference_steps=50,
            guidance_scale=7.5,
            negative_prompt=make_negative_prompt(NEGATIVE_PROMPT),
            cross_attention_kwargs={"scale": 0.75},
        ).images[0]

        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        return {"image": img_str}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An error occurred during image processing: {str(e)}")

if __name__ == "__main__":


    nest_asyncio.apply()

    ngrok.set_auth_token("ngrok token")
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")

    uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: NgrokTunnel: "https://f903-34-125-138-247.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [342]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     125.132.174.206:0 - "POST /fashion HTTP/1.1" 400 Bad Request
INFO:     125.132.174.206:0 - "POST /fashion HTTP/1.1" 400 Bad Request


/usr/local/lib/python3.10/dist-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


  0%|          | 0/50 [00:00<?, ?it/s]

...

INFO:     125.132.174.206:0 - "POST /fashion HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     125.132.174.206:0 - "POST /fashion HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     125.132.174.206:0 - "POST /fashion HTTP/1.1" 200 OK
